In [1]:
import deepquantum as dq
import deepquantum.photonic as dqp
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import numpy as np
from numpy import sin, cos, sinh, cosh, pi, nan

In [3]:
import numpy as np

array = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])

array[[0, 0, -1, -1], [0, -1, 0, -1]] = [[1,1,1,1]]
array

array([[1, 2, 1],
       [4, 5, 6],
       [1, 8, 1]])

In [4]:
class QumodeWithLoss():
    def __init__(self, circuit, loss, params, order):
        self.layers = len(circuit[0])
        self.n_modes = len(circuit)
        self.circuit = circuit
        self.loss = loss
        self.params = params
        self.loss_params = np.log(10) / 10
        self.order = order

        print('layers:', self.layers)
        print('n_modes:', self.n_modes)

    def get_matrix(self, type, params, dimention, position):

        def bs_matrix(eta):
            matrix = np.eye(dimention, dtype=complex)
            phi = pi
            theta = pi/2 - eta
            bs =  np.array([[cos(theta), -cos(phi)*sin(theta),           0, -sin(phi)*sin(theta)],
                            [cos(phi)*sin(theta),  cos(theta), -sin(phi)*sin(theta),           0],
                            [         0,  sin(phi)*sin(theta),  cos(theta), -cos(phi)*sin(theta)],
                            [sin(phi)*sin(theta),           0,  cos(phi)*sin(theta),  cos(theta)]])
            # matrix[2*position+1:2*position+3, 2*position+1:2*position+3] = bs
            matrix[position][position] = bs[0,0]
            matrix[position][position+1] = bs[0,1]
            matrix[position][position+self.n_modes] = bs[0,2]
            matrix[position][position+self.n_modes+1] = bs[0,3]
            matrix[position+1][position] = bs[1,0]
            matrix[position+1][position+1] = bs[1,1]
            matrix[position+1][position+self.n_modes] = bs[1,2]
            matrix[position+1][position+self.n_modes+1] = bs[1,3]
            matrix[position+self.n_modes][position] = bs[2,0]
            matrix[position+self.n_modes][position+1] = bs[2,1]
            matrix[position+self.n_modes][position+self.n_modes] = bs[2,2]
            matrix[position+self.n_modes][position+self.n_modes+1] = bs[2,3]
            matrix[position+self.n_modes+1][position] = bs[3,0]
            matrix[position+self.n_modes+1][position+1] = bs[3,1]
            matrix[position+self.n_modes+1][position+self.n_modes] = bs[3,2]
            matrix[position+self.n_modes+1][position+self.n_modes+1] = bs[3,3]
            return matrix

        def s_matrix(xi):
            r = xi[0]
            theta = xi[1]
            matrix = np.eye(dimention, dtype=complex)
            s = np.array([[cosh(r)-cos(theta)*sinh(r), -sin(theta)*sinh(r)],
                            [-sin(theta)*sinh(r), cosh(r)+cos(theta)*sinh(r)]])
            # matrix[2*position:2*position+2, 2*position:2*position+2] = s
            matrix[position][position] = s[0,0]
            matrix[position][position+self.n_modes] = s[0,1]
            matrix[position+self.n_modes][position] = s[1,0]
            matrix[position+self.n_modes][position+self.n_modes] = s[1,1]
            return matrix
        
        def s2_matrix(xi):
            r = xi[0]
            theta = xi[1]
            matrix = np.eye(dimention, dtype=complex)
            s2 = np.array([[cosh(r), cos(theta)*sinh(r), 0, sin(theta)*sinh(r)],
                            [cos(theta)*sinh(r), cosh(r), sin(theta)*sinh(r), 0],
                            [0, sin(theta)*sinh(r), cosh(r), -cos(theta)*sinh(r)],
                            [sin(theta)*sinh(r), 0, -cos(theta)*sinh(r), cosh(r)]])
            # matrix[2*position:2*position+4, 2*position:2*position+4] = s2
            matrix[position][position] = s2[0,0]
            matrix[position][position+self.n_modes] = s2[0,2]
            matrix[position][position+1] = s2[0,1]
            matrix[position][position+self.n_modes+1] = s2[0,3]
            matrix[position+self.n_modes][position] = s2[2,0]
            matrix[position+self.n_modes][position+self.n_modes] = s2[2,2]
            matrix[position+self.n_modes][position+1] = s2[2,1]
            matrix[position+self.n_modes][position+self.n_modes+1] = s2[2,3]
            matrix[position+1][position] = s2[1,0]
            matrix[position+1][position+self.n_modes] = s2[1,2]
            matrix[position+1][position+1] = s2[1,1]
            matrix[position+1][position+self.n_modes+1] = s2[1,3]
            matrix[position+self.n_modes+1][position] = s2[3,0]
            matrix[position+self.n_modes+1][position+self.n_modes] = s2[3,2]
            matrix[position+self.n_modes+1][position+1] = s2[3,1]
            matrix[position+self.n_modes+1][position+self.n_modes+1] = s2[3,3]
            return matrix
        
        def ps_matrix(theta):
            matrix = np.eye(dimention, dtype=complex)
            ps = np.array([[cos(theta), -sin(theta)],
                            [sin(theta), cos(theta)]])
            # matrix[2*position+1, 2*position+1] = ps
            matrix[position, position] = ps[0,0]
            matrix[position, position+self.n_modes] = ps[0,1]
            matrix[position+self.n_modes, position] = ps[1,0]
            matrix[position+self.n_modes, position+self.n_modes] = ps[1,1]

            return matrix

        if type == 'BS' :
            # print('BS', bs_matrix(params).real)
            return bs_matrix(params)
        elif type == 'S' :
            # print('S', s_matrix(params).real)
            return s_matrix(params)
        elif type == 'S2' :
            # print('S2', s2_matrix(params).real)
            return s2_matrix(params)
        elif type == 'PS' :
            # print('PS', ps_matrix(params).real)
            return ps_matrix(params)
        else:
            return np.eye(dimention, dtype=complex)

    def forward(self):
        cov = np.eye(self.n_modes*2, dtype=complex)
        for i in range(self.layers):
            for j in range(self.n_modes):
                type = self.order[str(self.circuit[j][i])]
                params = self.params[j][i]
                matrix = self.get_matrix(type=type, params=params, 
                                    dimention=self.n_modes*2, 
                                    position=j)
                cov = matrix @ cov @ matrix.conj().T

        return cov

In [5]:
import pandas as pd

order = {'0': 'NULL', 
         '1': 'BS', '2': 'PS', 
         '3': 'S', '4': 'S2'}

layers = 1
n_modes = 4

# circuit = np.zeros((layers, n_modes))
# circuit_origin = np.array([[4, 0, 0, 0, 0, 0],
#                            [4, 1, 2, 0, 1, 2],
#                            [3, 1, 1, 2, 1, 0],
#                            [3, 0, 1, 0, 0, 0],])
# circuit = np.array([[4, 0, 0, 0, 0, 0],
#                     [0, 1, 2, 0, 1, 2],
#                     [3, 0, 1, 2, 0, 0],
#                     [3, 0, 0, 0, 0, 0],])
# loss = np.ones((layers, n_modes))
# q = nan
# params = [[[1,1], q, q, q, q, q],
#           [[q,q], 1, 1, q, 1, 1],
#           [[1,1], q, 1, 1, q, q],
#           [[1,1], q, q, q, q, q],]

q = nan
circuit = np.array([[4, 1, 2],
                    [0, 0, 2],
                    [3, 1, 2],
                    [0, 0, 2],])
params = [[[1,1], 1, pi],
            [q, q, 0],
            [[1,1], 4, pi],
            [q, q, 0]]
loss = np.ones((n_modes, layers))
            

a = QumodeWithLoss(circuit, loss, params, order)
pd.DataFrame(a.forward().real)

layers: 3
n_modes: 4


,0,1,2,3,4,5,6,7
0,5.544056,-0.815482,0.000000,0.000000,2.775083,-1.270038,0.000000,0.000000
1,-0.815482,1.980336,0.000000,0.000000,-1.270038,-2.775083,0.000000,0.000000
2,0.000000,0.000000,1.459686,0.397027,0.000000,0.000000,-1.747975,-1.509710
3,0.000000,0.000000,0.397027,1.342909,0.000000,0.000000,-1.509710,-1.303923
4,2.775083,-1.270038,0.000000,0.000000,1.980336,0.815482,0.000000,0.000000
5,-1.270038,-2.775083,0.000000,0.000000,0.815482,5.544056,0.000000,0.000000
6,0.000000,0.000000,-1.747975,-1.509710,0.000000,0.000000,3.704409,2.335774
7,0.000000,0.000000,-1.509710,-1.303923,0.000000,0.000000,2.335774,3.017388


In [6]:
import CM.covariance as CV

Ns: int = 1
Nm: int = 3
NL: int = 4

eta: float = 1

s_r: float = 1
s_phi: float = 1

delta_k: float = 1
r_k: float = 1
phi_k: float = 1

b = CV.TripartiteMutualInformation(
    Ns=Ns, Nm=Nm, NL=NL,
    eta=eta,
    s_r=s_r, s_phi=s_phi,
    delta_k=delta_k, r_k=r_k, phi_k=phi_k
)

In [7]:
pd.DataFrame(b.get_cov())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3.762196e+00,-0.029839,0.705298,0.178703,1.722900,-2.151912,0.304521,0.048752,4.003104e-17,-0.026003,-0.205246,-0.604110,1.994811,0.306748,-0.665391,0.075926
1,-2.983915e-02,0.471405,-0.006764,-0.000030,-0.024136,0.021613,-0.001162,-0.000742,-2.600326e-02,-1.523706,0.004301,0.006875,-0.015649,-0.009776,0.007901,-0.000647
2,7.052978e-01,-0.006764,3.746803,-0.107059,0.486399,-0.064054,-0.111733,0.017547,-2.052460e-01,0.004301,3.478056,-0.069413,-0.220006,0.435552,-0.097369,-0.005106
3,1.787035e-01,-0.000030,-0.107059,6.967034,0.066617,0.284648,-0.122045,0.004446,-6.041098e-01,0.006875,-0.069413,-1.449599,-0.452974,0.248056,0.035516,-0.015029
4,1.722900e+00,-0.024136,0.486399,0.066617,2.472081,-1.516803,0.150855,0.042863,1.994811e+00,-0.015649,-0.220006,-0.452974,-1.124268,0.441399,-0.509969,0.049628
5,-2.151912e+00,0.021613,-0.064054,0.284648,-1.516803,3.102928,0.286665,-0.053536,3.067476e-01,-0.009776,0.435552,0.248056,0.441399,2.228665,0.331906,0.007631
6,3.045214e-01,-0.001162,-0.111733,-0.122045,0.150855,0.286665,7.206595,0.007576,-6.653915e-01,0.007901,-0.097369,0.035516,-0.509969,0.331906,-0.490987,-0.016554
7,4.875157e-02,-0.000742,0.017547,0.004446,0.042863,-0.053536,0.007576,1.803808,7.592607e-02,-0.000647,-0.005106,-0.015029,0.049628,0.007631,-0.016554,-3.050009
8,4.003104e-17,-0.026003,-0.205246,-0.604110,1.994811,0.306748,-0.665391,0.075926,3.762196e+00,0.029839,-0.705298,-0.178703,-1.722900,2.151912,-0.304521,-0.048752
9,-2.600326e-02,-1.523706,0.004301,0.006875,-0.015649,-0.009776,0.007901,-0.000647,2.983915e-02,7.052987,0.006764,0.000030,0.024136,-0.021613,0.001162,0.000742


In [8]:
pd.DataFrame(b.get_sigma_sm2())

,0,1,2,3,4,5
0,3.762196e+00,0.705298,0.178703,4.003104e-17,-0.205246,-0.604110
1,7.052978e-01,3.746803,-0.107059,-2.052460e-01,3.478056,-0.069413
2,1.787035e-01,-0.107059,6.967034,-6.041098e-01,-0.069413,-1.449599
3,4.003104e-17,-0.205246,-0.604110,3.762196e+00,-0.705298,-0.178703
4,-2.052460e-01,3.478056,-0.069413,-7.052978e-01,3.777589,0.107059
5,-6.041098e-01,-0.069413,-1.449599,-1.787035e-01,0.107059,0.557357


In [9]:
import pandas as pd

order = {'0': 'NULL', 
         '1': 'BS', '2': 'PS', 
         '3': 'S', '4': 'S2'}
q = nan

Ns: int = 1
Nm: int = 2
NL: int = 5

eta: float = 1

s_r: float = 1
s_phi: float = 1

delta_k: float = 1
r_k: float = 1
phi_k: float = 1


n_modes = Ns + Nm + NL
layers = 4*(Nm+NL) - 5
circuit = np.zeros((n_modes, layers), dtype=int)
circuit_origin = np.zeros((n_modes, layers), dtype=int)
loss = np.zeros((n_modes, layers))
params = []
for i in range(n_modes):
    params.append([q]*layers)
    
# S2
circuit[0][0] = 4
params[0][0] = [s_r, s_phi]
circuit_origin[0][0] = 4
circuit_origin[1][0] = 4

# S
for i in range(2, Ns+Nm+NL):
    circuit[i][0] = 3
    params[i][0] = [r_k, phi_k]
    circuit_origin[i][0] = 3

# BS+PS
for i in range(1,Ns+Nm+NL):
    for j in range(Nm+NL-i):
        circuit[i][2*i-1+4*j] = 1
        circuit[i][2*i+4*j] = 2
        params[i][2*i-1+4*j] = eta
        params[i][2*i+4*j] = delta_k
        circuit_origin[i][2*i-1+4*j] = 1
        circuit_origin[i+1][2*i-1+4*j] = 1
        circuit_origin[i][2*i+4*j] = 2


a = QumodeWithLoss(circuit, loss, params, order)
pd.DataFrame(a.forward().real)
# print('circuit:\n', circuit)
# print('circuit_origin:\n', circuit_origin)
# print('params:\n', params)
# print('loss:\n', loss)


layers: 23
n_modes: 8


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3.762196e+00,-0.029839,0.705298,0.178703,1.722900,-2.151912,0.304521,0.048752,4.003104e-17,-0.026003,-0.205246,-0.604110,1.994811,0.306748,-0.665391,0.075926
1,-2.983915e-02,0.471405,-0.006764,-0.000030,-0.024136,0.021613,-0.001162,-0.000742,-2.600326e-02,-1.523706,0.004301,0.006875,-0.015649,-0.009776,0.007901,-0.000647
2,7.052978e-01,-0.006764,3.746803,-0.107059,0.486399,-0.064054,-0.111733,0.017547,-2.052460e-01,0.004301,3.478056,-0.069413,-0.220006,0.435552,-0.097369,-0.005106
3,1.787035e-01,-0.000030,-0.107059,6.967034,0.066617,0.284648,-0.122045,0.004446,-6.041098e-01,0.006875,-0.069413,-1.449599,-0.452974,0.248056,0.035516,-0.015029
4,1.722900e+00,-0.024136,0.486399,0.066617,2.472081,-1.516803,0.150855,0.042863,1.994811e+00,-0.015649,-0.220006,-0.452974,-1.124268,0.441399,-0.509969,0.049628
5,-2.151912e+00,0.021613,-0.064054,0.284648,-1.516803,3.102928,0.286665,-0.053536,3.067476e-01,-0.009776,0.435552,0.248056,0.441399,2.228665,0.331906,0.007631
6,3.045214e-01,-0.001162,-0.111733,-0.122045,0.150855,0.286665,7.206595,0.007576,-6.653915e-01,0.007901,-0.097369,0.035516,-0.509969,0.331906,-0.490987,-0.016554
7,4.875157e-02,-0.000742,0.017547,0.004446,0.042863,-0.053536,0.007576,1.803808,7.592607e-02,-0.000647,-0.005106,-0.015029,0.049628,0.007631,-0.016554,-3.050009
8,4.003104e-17,-0.026003,-0.205246,-0.604110,1.994811,0.306748,-0.665391,0.075926,3.762196e+00,0.029839,-0.705298,-0.178703,-1.722900,2.151912,-0.304521,-0.048752
9,-2.600326e-02,-1.523706,0.004301,0.006875,-0.015649,-0.009776,0.007901,-0.000647,2.983915e-02,7.052987,0.006764,0.000030,0.024136,-0.021613,0.001162,0.000742


In [10]:
Ns: int = 1
Nm: int = 2
NL: int = 1
eta: float = 1
delta_k: float = 1
r_k: float = 1
phi_k: float = 1
shots: int = 10000
init_state: any = 'vac'
s_r: float = 1
s_phi: float = 1
m1_r: float = 0
m1_phi: float = 0

N_total = Ns + Nm + NL
theta = 2*eta - np.pi

cir = dq.QumodeCircuit(nmode=N_total, init_state=init_state, 
                    backend='gaussian', 
                    name='try', noise=False,
                    mu = 0, sigma = 0)

# s0,m1, two-mode squeezed vacuum (TMSV) state 
# cir.ps(0, [2])
cir.s2(wires=[0, 1], r=1, theta=1)


eta = 1
theta = 2*eta - np.pi
# cir.bs_ry(wires=[0, 1], inputs=theta)
cir.bs(wires=[0, 1], inputs=[np.pi/2-eta, np.pi])
cir.s(wires=2, r=1, theta=1)

eta = 4
theta = 2*eta - np.pi
cir.bs_ry(wires=[2, 3], inputs=theta)

delta_k = [pi, 0, pi, 0]
for i in range(Ns+Nm+NL):
    cir.ps(wires=i, inputs=delta_k[i])

state = cir.forward()


#线路可视化
cir.draw('pic/CM_circ_num_ME{}.svg'.format(N_total))

In [11]:
pd.DataFrame(state[0][0])

,0,1,2,3,4,5,6,7
0,5.544056,-0.815482,0.000000,0.000000,2.775083,-1.270038,0.000000,0.000000
1,-0.815482,1.980336,0.000000,0.000000,-1.270038,-2.775083,0.000000,0.000000
2,0.000000,0.000000,1.459686,0.397027,0.000000,0.000000,-1.747975,-1.509710
3,0.000000,0.000000,0.397027,1.342908,0.000000,0.000000,-1.509710,-1.303923
4,2.775083,-1.270038,0.000000,0.000000,1.980336,0.815482,0.000000,0.000000
5,-1.270038,-2.775083,0.000000,0.000000,0.815482,5.544056,0.000000,0.000000
6,0.000000,0.000000,-1.747975,-1.509710,0.000000,0.000000,3.704409,2.335774
7,0.000000,0.000000,-1.509710,-1.303923,0.000000,0.000000,2.335774,3.017387


In [12]:
Ns: int = 1
Nm: int = 2
NL: int = 5

eta: float = 1

delta_k: float = 1
r_k: float = 1
phi_k: float = 1

shots: int = 10000
init_state: any = 'vac'

s_r: float = 1
s_phi: float = 1

N_total = Ns + Nm + NL
theta = 2*eta - np.pi

cir = dq.QumodeCircuit(nmode=N_total, init_state=init_state, 
                    backend='gaussian', 
                    name='try', noise=False,
                    mu = 0, sigma = 0)

# s0,m1, two-mode squeezed vacuum (TMSV) state 

cir.s2(wires=[0, 1], r=s_r, theta=s_phi)

for i in range(Nm+NL-1):
    cir.s(wires=2+i, r=r_k, theta=phi_k)

for i in range(N_total-2):
    for j in range(N_total-i-2):
        cir.bs_ry(wires=[j+1,j+2], inputs=theta)
        cir.ps(j+1, [delta_k])

state = cir.forward()
print(state[0])
print(state[1])

#线路可视化
cir.draw('pic/CM_circ_num_ME{}.svg'.format(N_total))

tensor([[[ 3.7622e+00, -2.9839e-02,  7.0530e-01,  1.7870e-01,  1.7229e+00,
          -2.1519e+00,  3.0452e-01,  4.8752e-02,  1.1404e-08, -2.6003e-02,
          -2.0525e-01, -6.0411e-01,  1.9948e+00,  3.0675e-01, -6.6539e-01,
           7.5926e-02],
         [-2.9839e-02,  4.7140e-01, -6.7642e-03, -3.0465e-05, -2.4136e-02,
           2.1613e-02, -1.1619e-03, -7.4235e-04, -2.6003e-02, -1.5237e+00,
           4.3014e-03,  6.8750e-03, -1.5649e-02, -9.7759e-03,  7.9007e-03,
          -6.4692e-04],
         [ 7.0530e-01, -6.7642e-03,  3.7468e+00, -1.0706e-01,  4.8640e-01,
          -6.4054e-02, -1.1173e-01,  1.7547e-02, -2.0525e-01,  4.3012e-03,
           3.4781e+00, -6.9413e-02, -2.2001e-01,  4.3555e-01, -9.7369e-02,
          -5.1062e-03],
         [ 1.7870e-01, -3.0465e-05, -1.0706e-01,  6.9670e+00,  6.6617e-02,
           2.8465e-01, -1.2204e-01,  4.4458e-03, -6.0411e-01,  6.8746e-03,
          -6.9413e-02, -1.4496e+00, -4.5297e-01,  2.4806e-01,  3.5516e-02,
          -1.5029e-02],
    

In [13]:
pd.DataFrame(state[0][0].real)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3.762196e+00,-0.029839,0.705298,0.178704,1.722901,-2.151912,0.304521,0.048752,1.140361e-08,-0.026003,-0.205246,-0.604110,1.994811,0.306748,-0.665391,0.075926
1,-2.983915e-02,0.471404,-0.006764,-0.000030,-0.024136,0.021613,-0.001162,-0.000742,-2.600336e-02,-1.523704,0.004301,0.006875,-0.015649,-0.009776,0.007901,-0.000647
2,7.052977e-01,-0.006764,3.746804,-0.107060,0.486399,-0.064054,-0.111733,0.017547,-2.052462e-01,0.004301,3.478056,-0.069413,-0.220006,0.435552,-0.097369,-0.005106
3,1.787036e-01,-0.000030,-0.107060,6.967034,0.066617,0.284649,-0.122045,0.004446,-6.041104e-01,0.006875,-0.069413,-1.449599,-0.452974,0.248056,0.035516,-0.015029
4,1.722901e+00,-0.024136,0.486399,0.066617,2.472082,-1.516803,0.150855,0.042863,1.994811e+00,-0.015649,-0.220006,-0.452975,-1.124267,0.441399,-0.509969,0.049628
5,-2.151912e+00,0.021613,-0.064054,0.284649,-1.516803,3.102928,0.286664,-0.053536,3.067477e-01,-0.009776,0.435551,0.248056,0.441399,2.228666,0.331906,0.007631
6,3.045213e-01,-0.001162,-0.111733,-0.122045,0.150855,0.286664,7.206596,0.007576,-6.653913e-01,0.007901,-0.097369,0.035516,-0.509969,0.331906,-0.490988,-0.016553
7,4.875155e-02,-0.000742,0.017547,0.004446,0.042863,-0.053536,0.007576,1.803808,7.592605e-02,-0.000647,-0.005106,-0.015029,0.049627,0.007631,-0.016554,-3.050009
8,1.140361e-08,-0.026003,-0.205246,-0.604110,1.994811,0.306748,-0.665391,0.075926,3.762196e+00,0.029839,-0.705298,-0.178704,-1.722901,2.151912,-0.304521,-0.048752
9,-2.600329e-02,-1.523704,0.004301,0.006875,-0.015649,-0.009776,0.007901,-0.000647,2.983916e-02,7.052988,0.006765,0.000030,0.024136,-0.021613,0.001162,0.000742
